## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [ ]:
use_hydroelastic = False # this takes forever

scenario = load_scenario()

# since makehardwarestation finalizes the plant before giving it back to us
def callback(parser: Parser):
    plant = parser.plant()
    plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
    scene_graph: SceneGraph = parser.scene_graph()
    config = scene_graph.get_config()
    if use_hydroelastic:
        config.default_proximity_properties.compliance_type = "compliant"
    scene_graph.set_config(config)

station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
plant: MultibodyPlant = station.plant()


builder = DiagramBuilder()
builder.AddSystem(station)

to_point_cloud = AddPointClouds(
    scenario=scenario, station=station, builder=builder, meshcat=meshcat
)
builder.ExportOutput(
    to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
)

# diagram = builder.Build()


OutputPortIndex(0)

In [ ]:
from scripts.grasp_broom import plan_path, build_temp_plant

# basic test
# X_WG1 = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 0.5, 1])
# X_WG2 = RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 1, 1])

# meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("start", X_WG1)
# meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("goal", X_WG2)


In [ ]:
# test function to see the temp plant

# diagram, plant, gripper_frame = build_temp_plant(q0 = None, meshcat = meshcat)
# config = VisualizationConfig(lcm_bus='default', publish_period=0.015625, publish_illustration=True, default_illustration_color=Rgba(r=0.9, g=0.9, b=0.9, a=1.0), publish_proximity=True, default_proximity_color=Rgba(r=0.8, g=0.0, b=0.0, a=1.0), initial_proximity_alpha=0.5, publish_contacts=True, publish_inertia=True, enable_meshcat_creation=True, delete_on_initialization_event=True, enable_alpha_sliders=False)
# context = diagram.CreateDefaultContext()
# diagram.ForcedPublish(context)


In [7]:
path = plan_path(X_WG1, X_WG2)

7
Collision optimization failed


In [ ]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource

# grasp broom
X_WStart = RigidTransform([0, 1.0, 0.75])
X_WGoal = RigidTransform([0.6, 2.0, 0.5])

traj = plan_path(X_WStart, X_WGoal)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t)}')

# add traj to builder and connect
q_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
wsg_src = builder.AddSystem(ConstantVectorSource([1.0]))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()


7
Collision optimization failed
time0.0: [[-0.04590995]
 [ 0.08021216]
 [-0.0329916 ]
 [-1.1327646 ]
 [-0.00571152]
 [ 1.50193479]
 [-0.03695524]]
time0.07869913379388464: [[-0.08744854]
 [ 0.10507846]
 [-0.01385449]
 [-1.09836256]
 [-0.02537265]
 [ 1.52309157]
 [-0.03142739]]
time0.15739826758776929: [[-0.18742371]
 [ 0.15301554]
 [ 0.02091897]
 [-1.03362918]
 [-0.06291499]
 [ 1.5608398 ]
 [-0.01838708]]
time0.23609740138165392: [[-0.30902716]
 [ 0.18426834]
 [ 0.03761422]
 [-0.99593344]
 [-0.08637325]
 [ 1.57686827]
 [-0.00312898]]
time0.31479653517553857: [[-0.42874705]
 [ 0.17978295]
 [ 0.02361881]
 [-1.01300578]
 [-0.08088713]
 [ 1.55656251]
 [ 0.01081237]]
time0.39349566896942323: [[-0.54649779]
 [ 0.15697757]
 [ 0.00349881]
 [-1.06035853]
 [-0.0617325 ]
 [ 1.52705779]
 [ 0.02300347]]
time0.47219480276330783: [[-0.66447047]
 [ 0.13603186]
 [ 0.00450215]
 [-1.10907873]
 [-0.0465435 ]
 [ 1.51821465]
 [ 0.0334246 ]]
time0.5508939365571925: [[-0.7830444 ]
 [ 0.12115012]
 [ 0.03535715

In [8]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()